In [2]:
import pandas as pd
import numpy as np
from string import Template
import random

In [8]:
!tree ..

..
|-- input
|   `-- train_hwu.csv
|-- lib
|   `-- kaggle
|       `-- gcp.py
`-- working

4 directories, 2 files


In [9]:
df = pd.read_csv("../input/train_hwu.csv")

In [10]:
df

,text,category
0,what alarms do i have set right now,alarm_query
1,checkout today alarm of meeting,alarm_query
2,report alarm settings,alarm_query
3,see see for me the alarms that you have set to...,alarm_query
4,is there an alarm for ten am,alarm_query
...,...,...
9955,how hot is it in miami,weather_query
9956,will it snow next week,weather_query
9957,am i gonna need rain boots,weather_query
9958,should i bring warm clothes,weather_query


In [11]:
instruct_dict = {
 "Definitions": [
                    "You will be given some input text and you need to classify the intent of the text.",
                    "Given an input, classify the intent of the input text.",
                    "Select the correct intent of the given input text from a list of options.",
                    ],
"Intent Descriptions": [
    {
    "intent": "alarm_remove",
    "description": "an intent or action related to removing alarms"
    },
    {
    "intent": "calendar_set",
    "description": "an intent related to setting or scheduling events, appointments, reminders, or tasks on a calendar"
    },
    {
    "intent": "cooking_recipe",
    "description": "an intent related to outlining the steps, purpose or goal behind creating a particular dish"
    },
    {
    "intent": "datetime_convert",
    "description": "an intent related to converting dates and times from one format or timezone to another"
    },
    {
    "intent": "general_confirm",
    "description": "an intent related to a user's action or request to confirm a previous statement, action, or decision"
    },
    {
    "intent": "iot_coffee",
    "description": "an intent that pertains to an action or command related to an Internet of Things (IoT) enabled coffee machine"
    },
    {
    "intent": "music_query",
    "description": "an intent related to a user's action or request seeking information or assistance related to music"
    },
    {
    "intent": "qa_currency",
    "description": "an intent related to a question-and-answer (QA) scenario related to currency"
    },
],
"Positive Examples" : [
    {
        "input": "what alarms do i have set right now.",
        "outputs": [
                    "alarm_query"
                    ],
        "index": 0,
    },
    {
        "input": "go to youtube and play me the best wedding songs collection.",
        "outputs": [
                    "play_music"
                    ],
        "index": 6769,
    },
    {
        "input": "please play next episode.",
        "outputs": [
                    "iot_hue_lightup"
                    ],
        "index": 6784,
    },
    {
        "input": "turn on flappy bird.",
        "outputs": [
                    "play_game"
                    ],
        "index": 6447,
    }, 
    {
        "input": "define god.",
        "outputs": [
                    "qa_definition"
                    ],
        "index": 7422,
    },
    {
        "input": "twenty minus ten.",
        "outputs": [
                    "qa_maths"
                    ],
        "index": 7781,
    },
    {
        "input": "give me the opening stock price of infosys technologies today.",
        "outputs": [
                    "qa_stock"
                    ],
        "index": 7796,
    },
    {
        "input": "can i order takeaway from spanish place.",
        "outputs": [
                    "takeaway_order"
                    ],
        "index": 8832,
    },
    {
        "input": "please book me a taxi uber.",
        "outputs": [
                    "transport_taxi"
                    ],
        "index": 9354,
    }
]
}    

In [12]:
intent_classes = ["alarm_query", "alarm_remove", "alarm_set", "audio_volume_down", "audio_volume_mute", "audio_volume_up", "calendar_query", "calendar_remove", "calendar_set", "cooking_recipe", "datetime_convert", "datetime_query", "email_addcontact", "email_query", "email_querycontact", "email_sendemail", "general_affirm", "general_commandstop", "general_confirm", "general_dontcare", "general_explain", "general_joke", "general_negate", "general_praise", "general_quirky", "general_repeat", "iot_cleaning", "iot_coffee", "iot_hue_lightchange", "iot_hue_lightdim", "iot_hue_lightoff", "iot_hue_lighton", "iot_hue_lightup", "iot_wemo_off", "iot_wemo_on", "lists_createoradd", "lists_query", "lists_remove", "music_likeness", "music_query", "music_settings", "news_query", "play_audiobook", "play_game", "play_music", "play_podcasts", "play_radio", "qa_currency", "qa_definition", "qa_factoid", "qa_maths", "qa_stock", "recommendation_events", "recommendation_locations", "recommendation_movies", "social_post", "social_query", "takeaway_order", "takeaway_query", "transport_query", "transport_taxi", "transport_ticket", "transport_traffic", "weather_query"]

In [13]:
def list_tostring(classes):
    assert type(classes) == list
    return '||||'.join(classes)

def list_tostring_with_comma(classes):
    assert type(classes) == list
    return ','.join(classes)

In [14]:
def examples_tostring(examples):
    examples_list = []
    for example in examples:
        examples_list.append("INPUT: " + example['input']  + " INTENT: " + list_tostring_with_comma(example['outputs']))
    return '\n'.join(examples_list)

def intent_description_tostring(intent_descriptions):
    intent_description_list = []
    for intent_description in intent_descriptions:
        intent_description_list.append("INTENT: " + intent_description['intent']  + " DESCRIPTION: " + intent_description['description'])
    return '\n'.join(intent_description_list)

In [15]:
definitions = instruct_dict['Definitions']
intent_descriptions = instruct_dict['Intent Descriptions']
intent_examples = instruct_dict['Positive Examples']
num_intent_descriptions = 2
num_intent_examples = 2

In [32]:
def generate_instruction(example):
    prompt = (f"Below is an instruction that describes a task, along with an input, intent descriptions and examples to provide further context. "
          "Write an output that appropriately completes the request.\n\n"
          "### Instruction:\n"
          f"{example['instruction']}\n\n"
          f"### Input: \n"
          f"{example['input']}\n\n"
          f"### Intent Descriptions: \n"
          f"{example['intent description']}\n\n"
          f"### Examples: \n"
          f"{example['examples']}\n\n"
          f"### Intent: \n"
          f"{example['intent']}")
    return prompt


In [46]:
def construct_instruction(dp):
    classes = intent_classes
    random.shuffle(classes)
    definition = random.choice(definitions)
    instruction = definition + ' The possible intents are: '+ list_tostring_with_comma(classes)
    
    intent_description_prompts = np.random.choice(intent_descriptions, num_intent_descriptions, replace=False)
    intent_descriptions_string = intent_description_tostring(intent_description_prompts)
    examples_prompts = np.random.choice(intent_examples, num_intent_examples, replace=False)
    examples_string = examples_tostring(examples_prompts)
    
    text = generate_instruction({
        'instruction': instruction,
        'input': dp['text'],
        'intent description': intent_descriptions_string,
        'examples': examples_string,
        'intent': dp['category']
    })

    return text

In [47]:
df_10 = df[:10]
df_10

,text,category
0,what alarms do i have set right now,alarm_query
1,checkout today alarm of meeting,alarm_query
2,report alarm settings,alarm_query
3,see see for me the alarms that you have set to...,alarm_query
4,is there an alarm for ten am,alarm_query
5,confirm the alarm time,alarm_query
6,show my alarms,alarm_query
7,at what time have you set alarm for me,alarm_query
8,please list active alarms,alarm_query
9,please tell me about upcoming alarms,alarm_query


In [48]:
df_10['instruction'] = df_10.apply(lambda x: construct_instruction(x), axis=1)

/tmp/ipykernel_34/2351067532.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_10['instruction'] = df_10.apply(lambda x: construct_instruction(x), axis=1)


In [49]:
df_10

,text,category,instruction
0,what alarms do i have set right now,alarm_query,"Below is an instruction that describes a task,..."
1,checkout today alarm of meeting,alarm_query,"Below is an instruction that describes a task,..."
2,report alarm settings,alarm_query,"Below is an instruction that describes a task,..."
3,see see for me the alarms that you have set to...,alarm_query,"Below is an instruction that describes a task,..."
4,is there an alarm for ten am,alarm_query,"Below is an instruction that describes a task,..."
5,confirm the alarm time,alarm_query,"Below is an instruction that describes a task,..."
6,show my alarms,alarm_query,"Below is an instruction that describes a task,..."
7,at what time have you set alarm for me,alarm_query,"Below is an instruction that describes a task,..."
8,please list active alarms,alarm_query,"Below is an instruction that describes a task,..."
9,please tell me about upcoming alarms,alarm_query,"Below is an instruction that describes a task,..."


In [50]:
df_10.to_csv('trial.csv')